# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from ate_models.ATE_BERT_Dropout_CNN_BiLSTM_Linear import ATE_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
BERT_FINE_TUNED_PATH = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [12]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear.csv'

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
best_accuracy = 0.0

In [15]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [16]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [17]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [18]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [19]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_CNN_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0542467832565308
Epoch: 0/2, Batch: 50/501, Loss: 0.5321171283721924
Epoch: 0/2, Batch: 100/501, Loss: 0.1747092455625534
Epoch: 0/2, Batch: 150/501, Loss: 0.08273252844810486
Epoch: 0/2, Batch: 200/501, Loss: 0.04575229063630104
Epoch: 0/2, Batch: 250/501, Loss: 0.08083061873912811
Epoch: 0/2, Batch: 300/501, Loss: 0.029120909050107002
Epoch: 0/2, Batch: 350/501, Loss: 0.02484729327261448
Epoch: 0/2, Batch: 400/501, Loss: 0.019432995468378067
Epoch: 0/2, Batch: 450/501, Loss: 0.03155076503753662
Epoch: 0/2, Batch: 500/501, Loss: 0.027576666325330734
Epoch: 1/2, Batch: 0/501, Loss: 0.01648276299238205
Epoch: 1/2, Batch: 50/501, Loss: 0.030881445854902267
Epoch: 1/2, Batch: 100/501, Loss: 0.02345050871372223
Epoch: 1/2, Batch: 150/501, Loss: 0.02026584930717945
Epoch: 1/2, Batch: 200/501, Loss: 0.01580442115664482
Epoch: 1/2, Batch: 250/501, Loss: 0.019574681296944618
Epoch: 1/2, Batch: 300/501, Loss: 0.0244335625320673
Epoch: 1/2, Batch: 350/5

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0872057676315308
Epoch: 0/2, Batch: 50/501, Loss: 0.5923060178756714
Epoch: 0/2, Batch: 100/501, Loss: 0.1414772868156433
Epoch: 0/2, Batch: 150/501, Loss: 0.06604524701833725
Epoch: 0/2, Batch: 200/501, Loss: 0.04842788726091385
Epoch: 0/2, Batch: 250/501, Loss: 0.027280062437057495
Epoch: 0/2, Batch: 300/501, Loss: 0.03161023557186127
Epoch: 0/2, Batch: 350/501, Loss: 0.023147720843553543
Epoch: 0/2, Batch: 400/501, Loss: 0.047526001930236816
Epoch: 0/2, Batch: 450/501, Loss: 0.015185251832008362
Epoch: 0/2, Batch: 500/501, Loss: 0.010246201418340206
Epoch: 1/2, Batch: 0/501, Loss: 0.020910557359457016
Epoch: 1/2, Batch: 50/501, Loss: 0.01642727293074131
Epoch: 1/2, Batch: 100/501, Loss: 0.017603376880288124
Epoch: 1/2, Batch: 150/501, Loss: 0.018378645181655884
Epoch: 1/2, Batch: 200/501, Loss: 0.011199269443750381
Epoch: 1/2, Batch: 250/501, Loss: 0.01900157332420349
Epoch: 1/2, Batch: 300/501, Loss: 0.02291872352361679
Epoch: 1/2, Batch: 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 5/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0972031354904175
Epoch: 0/2, Batch: 50/501, Loss: 0.5322515368461609
Epoch: 0/2, Batch: 100/501, Loss: 0.15784379839897156
Epoch: 0/2, Batch: 150/501, Loss: 0.08054593205451965
Epoch: 0/2, Batch: 200/501, Loss: 0.04545075446367264
Epoch: 0/2, Batch: 250/501, Loss: 0.038978997617959976
Epoch: 0/2, Batch: 300/501, Loss: 0.030718455091118813
Epoch: 0/2, Batch: 350/501, Loss: 0.027123937383294106
Epoch: 0/2, Batch: 400/501, Loss: 0.03204634413123131
Epoch: 0/2, Batch: 450/501, Loss: 0.02087046392261982
Epoch: 0/2, Batch: 500/501, Loss: 0.02437399886548519
Epoch: 1/2, Batch: 0/501, Loss: 0.022309063002467155
Epoch: 1/2, Batch: 50/501, Loss: 0.026360150426626205
Epoch: 1/2, Batch: 100/501, Loss: 0.01920643262565136
Epoch: 1/2, Batch: 150/501, Loss: 0.019399791955947876
Epoch: 1/2, Batch: 200/501, Loss: 0.024579161778092384
Epoch: 1/2, Batch: 250/501, Loss: 0.012040296569466591
Epoch: 1/2, Batch: 300/501, Loss: 0.03633512929081917
Epoch: 1/2, Batch: 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.114091157913208
Epoch: 0/2, Batch: 50/501, Loss: 0.5482444167137146
Epoch: 0/2, Batch: 100/501, Loss: 0.13233618438243866
Epoch: 0/2, Batch: 150/501, Loss: 0.0622732937335968
Epoch: 0/2, Batch: 200/501, Loss: 0.07565288245677948
Epoch: 0/2, Batch: 250/501, Loss: 0.0391237847507
Epoch: 0/2, Batch: 300/501, Loss: 0.021603580564260483
Epoch: 0/2, Batch: 350/501, Loss: 0.026238104328513145
Epoch: 0/2, Batch: 400/501, Loss: 0.018476370722055435
Epoch: 0/2, Batch: 450/501, Loss: 0.023417802527546883
Epoch: 0/2, Batch: 500/501, Loss: 0.02657862938940525
Epoch: 1/2, Batch: 0/501, Loss: 0.014550133608281612
Epoch: 1/2, Batch: 50/501, Loss: 0.009722390212118626
Epoch: 1/2, Batch: 100/501, Loss: 0.027243467047810555
Epoch: 1/2, Batch: 150/501, Loss: 0.017576318234205246
Epoch: 1/2, Batch: 200/501, Loss: 0.026014912873506546
Epoch: 1/2, Batch: 250/501, Loss: 0.032570838928222656
Epoch: 1/2, Batch: 300/501, Loss: 0.01274008210748434
Epoch: 1/2, Batch: 350/

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0790808200836182
Epoch: 0/2, Batch: 50/501, Loss: 0.4406585395336151
Epoch: 0/2, Batch: 100/501, Loss: 0.13892026245594025
Epoch: 0/2, Batch: 150/501, Loss: 0.08815524727106094
Epoch: 0/2, Batch: 200/501, Loss: 0.039858244359493256
Epoch: 0/2, Batch: 250/501, Loss: 0.02615087293088436
Epoch: 0/2, Batch: 300/501, Loss: 0.024503469467163086
Epoch: 0/2, Batch: 350/501, Loss: 0.03498710319399834
Epoch: 0/2, Batch: 400/501, Loss: 0.024822881445288658
Epoch: 0/2, Batch: 450/501, Loss: 0.01935102790594101
Epoch: 0/2, Batch: 500/501, Loss: 0.027249792590737343
Epoch: 1/2, Batch: 0/501, Loss: 0.04391122981905937
Epoch: 1/2, Batch: 50/501, Loss: 0.017189139500260353
Epoch: 1/2, Batch: 100/501, Loss: 0.017277754843235016
Epoch: 1/2, Batch: 150/501, Loss: 0.01237617339938879
Epoch: 1/2, Batch: 200/501, Loss: 0.01241897139698267
Epoch: 1/2, Batch: 250/501, Loss: 0.018637463450431824
Epoch: 1/2, Batch: 300/501, Loss: 0.03461555764079094
Epoch: 1/2, Batch: 3

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.994949,0.994949,0.620298,0.994949,0.364183,0.994949,0.387661,364.732296
1,0.994926,0.994926,0.459046,0.994926,0.356646,0.994926,0.372002,361.319139
2,0.995203,0.995203,0.740861,0.995203,0.348298,0.995203,0.361377,361.295934
3,0.995172,0.995172,0.450783,0.995172,0.335284,0.995172,0.336382,360.377202
4,0.994781,0.994781,0.582125,0.994781,0.369920,0.994781,0.392177,360.619777
5,0.994938,0.994938,0.542740,0.994938,0.348508,0.994938,0.360780,360.779933
6,0.995008,0.995008,0.455764,0.995008,0.368660,0.995008,0.387644,360.821728
7,0.994711,0.994711,0.452215,0.994711,0.345377,0.994711,0.354418,362.203332
8,0.994910,0.994910,0.520803,0.994910,0.359041,0.994910,0.375594,365.873135
9,0.995102,0.995102,0.460763,0.995102,0.353891,0.995102,0.368064,363.622211


In [21]:
results.to_csv(STATS_OUTPUT)